In [ ]:
import pandas as pd
import numpy as np
import datetime
import dateparser
import dateutil
from pymongo import MongoClient
from statistics import mode

In [ ]:
client = MongoClient()
bbdb = client["BailBondDB"]
coll = bbdb["bond_court_individuals"]

### Records Useable for Racial Disparity Analysis

People who either have no record, no race data, or have been in jail too long for the case we saw to be the reason they are in jail. These are not useable.

In [ ]:
no_record = coll.find(
    {"$or" : [
        { "$and" : [
            {"Jail Record" : False},
            {"Arrest Record" : False},
            {"Court Record" : False}
        ]},
        {"Days in Jail" : {"$gt" : 14}},
        {"Race" : None}
    ]}
)

In [ ]:
no_record.count()

People for whom corroborating record was found, who hadn't been in jail more than a week before bond court date, and who have race recorded.

In [ ]:
race_data = coll.find(
    {"$and" : [
        {"$or" : [
            {"Jail Record" : True},
            {"Arrest Record" : True},
            {"Court Record" : True}
        ]},
        {"$or" : [
            {"Days in Jail" : {"$lt" : 15}},
            {"Days in Jail" : None}
        ]},
        {"Race" : { "$ne" : None}}
    ]}
)

In [ ]:
race_data.count()

In [ ]:
race_coll = bbdb["race_analysis"]
for record in race_data:
    race_coll.insert_one(record)

In [ ]:
race_coll.find().count()

In [ ]:
all_records = coll.find()

In [ ]:
all_records.count()

In [ ]:
coll.find({"Highest Charge Class" : None, "Number of Charges" : 0}).count()

In [ ]:
coll.find({"Number of Charges" : 0}).count()

In [ ]:
coll.find({ "$or" : [
                {"Total Bond" : 0},
                {"Total Bond" : None}
        ]}
).count()

In [ ]:
people_w_record_no_charges = coll.find(
    { "$and" : [
        { "$or" : [
            {"Jail Record" : True},
            {"Arrest Record" : True},
            {"Court Record" : True}
        ]},
         {"Highest Charge Class" : None}
    ]}
)

In [ ]:
inex_missing_charges = []
for person in people_w_record_no_charges:
    inex_missing_charges.append(person["All Spellings"])
    inex_missing_charges.append(person["Charges"])
    inex_missing_charges.append(person["Jail Record"])
    inex_missing_charges.append(person["Arrest Record"])
    inex_missing_charges.append(person["Court Record"])
    inex_missing_charges.append(person["Arrest ID"])

In [ ]:
inex_missing_charges

### Investigating Racial Disparities

In [ ]:
days_1 = race_coll.find({"Days in Jail" : 1})
days_2 = race_coll.find({"Days in Jail" : 2})
days_3 = race_coll.find({"Days in Jail" : 3})
days_4 = race_coll.find({"Days in Jail" : 4})
days_5 = race_coll.find({"Days in Jail" : 5})
days_6 = race_coll.find({"Days in Jail" : 6})
days_7 = race_coll.find({"Days in Jail" : 7})

In [ ]:
print(days_1.count(), days_2.count(), days_3.count(), days_4.count(), days_5.count(), days_6.count(), days_7.count())

In [ ]:
b_7days = race_coll.find({"Days in Jail" : 7, "Race" : "BLACK"}).count()
nb_7days = race_coll.find({"Days in Jail" : 7, "Race" : {"$ne" : "BLACK"}}).count()
b = race_coll.find({"Race" : "BLACK"}).count()
nb = race_coll.find({"Race" : {"$ne" : "BLACK"}}).count()

In [ ]:
print(nb, b, nb_7days, b_7days)

In [ ]:
perc_init_black = (b/(b + nb))*100 #percent initial sample who are black
perc_jail_black = (b_7days/(nb_7days + b_7days))*100 #percent black in jail 7 days later

In [ ]:
print(perc_init_black, perc_jail_black)

In [ ]:
b_7days_misd = race_coll.find(
    {"$and" : [
        {"Race" : "BLACK"},
        {"$or" : [
            {"Highest Charge Class" : "CLASS A MISDEMEANOR"},
            {"Highest Charge Class" : "CLASS B MISDEMEANOR"},
            {"Highest Charge Class" : "CLASS C MISDEMEANOR"}
        ]},
        {"Still in Jail After a Week" : True}
    ]}
).count()
nb_7days_misd = race_coll.find(
    {"$and" : [
        {"Race" : {"$ne" : "BLACK"}},
        {"$or" : [
            {"Highest Charge Class" : "CLASS A MISDEMEANOR"},
            {"Highest Charge Class" : "CLASS B MISDEMEANOR"},
            {"Highest Charge Class" : "CLASS C MISDEMEANOR"}
        ]},
        {"Still in Jail After a Week" : True}
    ]}
).count()
b_misd = race_coll.find(
    {"$and" : [
        {"Race" : "BLACK"},
        {"$or" : [
            {"Highest Charge Class" : "CLASS A MISDEMEANOR"},
            {"Highest Charge Class" : "CLASS B MISDEMEANOR"},
            {"Highest Charge Class" : "CLASS C MISDEMEANOR"}
        ]}]
}).count()
nb_misd = race_coll.find(
    {"$and" :
        [{"Race" : {"$ne" : "BLACK"}},
        {"$or" : [
            {"Highest Charge Class" : "CLASS A MISDEMEANOR"},
            {"Highest Charge Class" : "CLASS B MISDEMEANOR"},
            {"Highest Charge Class" : "CLASS C MISDEMEANOR"}
        ]}]
}
).count()

b_7days_4 = race_coll.find({"Still in Jail After a Week" : True, "Race" : "BLACK", "Highest Charge Class" : "CLASS 4 FELONY"}).count()
nb_7days_4 = race_coll.find({"Still in Jail After a Week" : True, "Race" : {"$ne" : "BLACK"}, "Highest Charge Class" : "CLASS 4 FELONY"}).count()
b_4 = race_coll.find({"Race" : "BLACK", "Highest Charge Class" : "CLASS 4 FELONY"}).count()
nb_4 = race_coll.find({"Race" : {"$ne" : "BLACK"}, "Highest Charge Class" : "CLASS 4 FELONY"}).count()

b_7days_3 = race_coll.find({"Still in Jail After a Week" : True, "Race" : "BLACK", "Highest Charge Class" : "CLASS 3 FELONY"}).count()
nb_7days_3 = race_coll.find({"Still in Jail After a Week" : True, "Race" : {"$ne" : "BLACK"}, "Highest Charge Class" : "CLASS 3 FELONY"}).count()
b_3 = race_coll.find({"Race" : "BLACK", "Highest Charge Class" : "CLASS 3 FELONY"}).count()
nb_3 = coll.find({"Race" : {"$ne" : "BLACK"}, "Highest Charge Class" : "CLASS 3 FELONY"}).count()

b_7days_2 = race_coll.find({"Still in Jail After a Week" : True, "Race" : "BLACK", "Highest Charge Class" : "CLASS 2 FELONY"}).count()
nb_7days_2 = race_coll.find({"Still in Jail After a Week" : True, "Race" : {"$ne" : "BLACK"}, "Highest Charge Class" : "CLASS 2 FELONY"}).count()
b_2 = race_coll.find({"Race" : "BLACK", "Highest Charge Class" : "CLASS 2 FELONY"}).count()
nb_2 = race_coll.find({"Race" : {"$ne" : "BLACK"}, "Highest Charge Class" : "CLASS 2 FELONY"}).count()

b_7days_1 = race_coll.find({"Still in Jail After a Week" : True, "Race" : "BLACK", "Highest Charge Class" : "CLASS 1 FELONY"}).count()
nb_7days_1 = race_coll.find({"Still in Jail After a Week" : True, "Race" : {"$ne" : "BLACK"}, "Highest Charge Class" : "CLASS 1 FELONY"}).count()
b_1 = race_coll.find({"Race" : "BLACK", "Highest Charge Class" : "CLASS 1 FELONY"}).count()
nb_1 = race_coll.find({"Race" : {"$ne" : "BLACK"}, "Highest Charge Class" : "CLASS 1 FELONY"}).count()

b_7days_x = race_coll.find({"Still in Jail After a Week" : True, "Race" : "BLACK", "Highest Charge Class" : "CLASS X FELONY"}).count()
nb_7days_x = race_coll.find({"Still in Jail After a Week" : True, "Race" : {"$ne" : "BLACK"}, "Highest Charge Class" : "CLASS X FELONY"}).count()
b_x = race_coll.find({"Race" : "BLACK", "Highest Charge Class" : "CLASS X FELONY"}).count()
nb_x = race_coll.find({"Race" : {"$ne" : "BLACK"}, "Highest Charge Class" : "CLASS X FELONY"}).count()

b_7days_m = race_coll.find({"Still in Jail After a Week" : True, "Race" : "BLACK", "Highest Charge Class" : "CLASS M FELONY"}).count()
nb_7days_m = race_coll.find({"Still in Jail After a Week" : True, "Race" : {"$ne" : "BLACK"}, "Highest Charge Class" : "CLASS M FELONY"}).count()
b_m = race_coll.find({"Race" : "BLACK", "Highest Charge Class" : "CLASS M FELONY"}).count()
nb_m = race_coll.find({"Race" : {"$ne" : "BLACK"}, "Highest Charge Class" : "CLASS M FELONY"}).count()

In [ ]:
clmisd_perc_arrested = (b_misd/(b_misd + nb_misd))*100
cl4_perc_arrested = (b_4/(b_4 + nb_4))*100
cl3_perc_arrested = (b_3/(b_3 + nb_3))*100
cl2_perc_arrested = (b_2/(b_2 + nb_2))*100
cl1_perc_arrested = (b_1/(b_1 + nb_1))*100
clx_perc_arrested = (b_x/(b_x + nb_x))*100
clm_perc_arrested = (b_m/(b_m + nb_m))*100
clmisd_perc_jail = (b_7days_misd/(b_7days_misd + nb_7days_misd))*100
cl4_perc_jail = (b_7days_4/(b_7days_4 + nb_7days_4))*100
cl3_perc_jail = (b_7days_3/(b_7days_3 + nb_7days_3))*100
cl2_perc_jail = (b_7days_2/(b_7days_2 + nb_7days_2))*100
cl1_perc_jail = (b_7days_1/(b_7days_1 + nb_7days_1))*100
clx_perc_jail = (b_7days_x/(b_7days_x + nb_7days_x))*100
clm_perc_jail = (b_7days_m/(b_7days_m + nb_7days_m))*100

total_misd_arrested = b_misd + nb_misd
total_misd_jail = b_7days_misd + nb_7days_misd
total_4_arrested = b_4 + nb_4
total_4_jail = b_7days_4 + nb_7days_4
total_3_arrested = b_3 + nb_3
total_3_jail = b_7days_3 + nb_7days_3
total_2_arrested = b_2 + nb_2
total_2_jail = b_7days_2 + nb_7days_2
total_1_arrested = b_1 + nb_1
total_1_jail = b_7days_1 + nb_7days_1
total_x_arrested = b_x + nb_x
total_x_jail = b_7days_x + nb_7days_x
total_m_arrested = b_m + nb_m
total_m_jail = b_7days_m + nb_7days_m

head = ("Felony Class", "% Bl. Init Obs", "Total Init Obs", "% Bl. Jail 7 Days", "Total Jail 7 Days")
totals = [
    ("All Misdemeanors", clmisd_perc_arrested, total_misd_arrested, clmisd_perc_jail, total_misd_jail),
    ("Class 4 Felony", cl4_perc_arrested, total_4_arrested, cl4_perc_jail, total_4_jail),
    ("Class 3 Felony", cl3_perc_arrested, total_3_arrested, cl3_perc_jail, total_3_jail),
    ("Class 2 Felony", cl2_perc_arrested, total_2_arrested, cl2_perc_jail, total_2_jail),
    ("Class 1 Felony", cl1_perc_arrested, total_1_arrested, cl1_perc_jail, total_1_jail),
    ("Class X Felony", clx_perc_arrested, total_x_arrested, clx_perc_jail, total_x_jail),
    ("Class M Felony", clm_perc_arrested, total_m_arrested, clm_perc_jail, total_m_jail)
]
result = pd.DataFrame(totals, columns = head)

In [ ]:
result

In [ ]:
cl4_perc_arrested = (b_4/(b_4 + nb_4))*100
cl4_perc_jail = (b_7days_4/(b_7days_4 + nb_7days_4))*100
total_4_arrested = b_4 + nb_4
total_4_jail = b_7days_4 + nb_7days_4

cl23_perc_arrested = ((b_3 + b_2)/(b_3 + b_2 + nb_3 + nb_2))*100
cl23_perc_jail = ((b_7days_3 + b_7days_2)/(b_7days_3 + b_7days_2 + nb_7days_3 + nb_7days_2))*100
total_23_arrested = b_3 + b_2 + nb_3 + nb_2
total_23_jail = b_7days_3 + b_7days_2 + nb_7days_3 + nb_7days_2

cl1xm_perc_arrested = ((b_1 + b_x + b_m) / (b_1 + b_x + b_m + nb_1 + nb_x + nb_m))*100
cl1xm_perc_jail = ((b_7days_1 + b_7days_x + b_7days_m) / (b_7days_1 + b_7days_x + b_7days_m + nb_7days_1 + nb_7days_x + nb_7days_m))*100
total_1xm_arrested = b_1 + b_x + b_m + nb_1 + nb_x + nb_m
total_1xm_jail = b_7days_1 + b_7days_x + b_7days_m + nb_7days_1 + nb_7days_x + nb_7days_m

head = ("Felony Class", "% Bl. Init Obs", "Total Init Obs", "% Bl. Jail 7 Days", "Total Jail 7 Days")
totals = [
    ("All Misdemeanors", clmisd_perc_arrested, total_misd_arrested, clmisd_perc_jail, total_misd_jail),
    ("Class 4 Felony", cl4_perc_arrested, total_4_arrested, cl4_perc_jail, total_4_jail),
    ("Class 2 & 3 Felony", cl23_perc_arrested, total_23_arrested, cl23_perc_jail, total_23_jail),
    ("Class 1, X & M Felony", cl1xm_perc_arrested, total_1xm_arrested, cl1xm_perc_jail, total_1xm_jail)
]
result_grouped = pd.DataFrame(totals, columns = head)

In [ ]:
result_grouped

In [ ]:
head = ["Class of Highest Charge", "Initial Observed", "Still in Jail after 7 Days", "% in Jail after 7 Days" ]

totals = [
    ("All Misdemeanors", total_misd_arrested, total_misd_jail, (total_misd_jail/total_misd_arrested)*100),
    ("Class 4 Felony", total_4_arrested, total_4_jail, (total_4_jail/total_4_arrested)*100),
    ("Class 2 & 3 Felony", total_23_arrested, total_23_jail, (total_23_jail/total_23_arrested)*100),
    ("Class 1, X & M Felony", total_1xm_arrested, total_1xm_jail, (total_1xm_jail/total_1xm_arrested)*100)
]

result_total = pd.DataFrame(totals, columns = head)

In [ ]:
result_total

In [ ]:
result_total.to_csv("../Output/Total Number of People Staying in Jail by Class.csv", index = False)

In [ ]:
stillin_misd = race_coll.find(
    {"$and" : [
        {"Race" : "BLACK"},
        {"$or" : [
            {"Highest Charge Class" : "CLASS A MISDEMEANOR"},
            {"Highest Charge Class" : "CLASS B MISDEMEANOR"},
            {"Highest Charge Class" : "CLASS C MISDEMEANOR"}
        ]},
        {"Still in Jail After a Week" : True},
        {"No Bond" : False},
        {"Total Bond" : {"$ne" : None}}
    ]}
)

stillin_4 = race_coll.find(
    {"$and" : [
        {"Race" : "BLACK"},
        {"Highest Charge Class" : "CLASS 4 FELONY"},
        {"Still in Jail After a Week" : True},
        {"No Bond" : False},
        {"Total Bond" : {"$ne" : None}}
    ]}
)

stillin_23 = race_coll.find(
    {"$and" : [
        {"Race" : "BLACK"},
        {"$or" : [
            {"Highest Charge Class" : "CLASS 2 FELONY"},
            {"Highest Charge Class" : "CLASS 3 FELONY"}
        ]},
        {"Still in Jail After a Week" : True},
        {"No Bond" : False},
        {"Total Bond" : {"$ne" : None}}
    ]}
)

stillin_1xm = race_coll.find(
    {"$and" : [
        {"Race" : "BLACK"},
        {"$or" : [
            {"Highest Charge Class" : "CLASS 1 FELONY"},
            {"Highest Charge Class" : "CLASS X FELONY"},
            {"Highest Charge Class" : "CLASS M FELONY"}
        ]},
        {"Still in Jail After a Week" : True},
        {"No Bond" : False},
        {"Total Bond" : {"$ne" : None}}
    ]}
)

In [ ]:
jail_count_misd = stillin_misd.count()
jail_count_4 = stillin_4.count()
jail_count_23 = stillin_23.count()
jail_count_1xm = stillin_1xm.count()

In [ ]:
jail_count_4

In [ ]:
bonds_misd = []
for person in stillin_misd:
    bonds_misd.append(person["Total Bond"])
bonds_4 = []
for person in stillin_4:
    bonds_4.append(person["Total Bond"])
bonds_23 = []
for person in stillin_23:
    bonds_23.append(person["Total Bond"])
bonds_1xm = []
for person in stillin_1xm:
    bonds_1xm.append(person["Total Bond"])

still_in_jail_bonds = [
("All Misdemeanors", jail_count_misd, sum(bonds_misd)/jail_count_misd, mode(bonds_misd)),
("Class 4 Felonies", jail_count_4, sum(bonds_4)/jail_count_4, mode(bonds_4)),
("Class 2 & 3 Felonies", jail_count_23, sum(bonds_23)/jail_count_23, mode(bonds_23)),
("Class 1, X & M Felonies", jail_count_1xm, sum(bonds_1xm)/jail_count_1xm, mode(bonds_1xm))
]

head = ["Crime Class", "Total Still in Jail after 7 Days", "Average Bond for Still in Jail after 7 Days", "Most Common Bond for Still in Jail after 7 Days"]

injail_bonds = pd.DataFrame(still_in_jail_bonds, columns = head)

In [ ]:
injail_bonds

In [ ]:
injail_bonds.to_csv("../Output/Bonds for People in Jail.csv", index = False)

### Bail by Judge

In [ ]:
judge_data = coll.find(
    {"$and" : [
        {"$or" : [
            {"Jail Record" : True},
            {"Arrest Record" : True},
            {"Court Record" : True}
        ]},
        {"$or" : [
            {"Days in Jail" : {"$lt" : 15}},
            {"Days in Jail" : None}
        ]},
        {"Judge" : { "$ne" : None}},
        {"Highest Charge Class" : {"$ne" : None}}
    ]}
)

In [ ]:
judge_data.count()

In [ ]:
judge_dropdata = coll.find(
    {"$or" : [
        {"$and" : [
            {"Jail Record" : False},
            {"Arrest Record" : False},
            {"Court Record" : False}
        ]},
        {"Days in Jail" : {"$gt" : 14}},
        {"Judge" : None},
        {"Highest Charge Class" : None}
    ]}
)

In [ ]:
judge_dropdata.count()

In [ ]:
judge_coll.drop()

In [ ]:
judge_coll = bbdb["judge_analysis"]
for record in judge_data:
    judge_coll.insert_one(record)

In [ ]:
all_peeps = judge_coll.find()

In [ ]:
head = ["Judge", "Courthouse", "Bond", "Highest Charge Class"]
grid = []
for person in all_peeps:
    row = []
    row.append(person["Judge"])
    row.append(person["Courthouse"])
    row.append(person["Total Bond"])
    row.append(person["Highest Charge Class"])
    grid.append(row)

In [ ]:
judge_chart = pd.DataFrame(grid, columns = head)

In [ ]:
judge_chart

In [ ]:
judge_chart.dropna(inplace = True)

In [ ]:
judge_chart

In [ ]:
bonds_by_judge = judge_chart.groupby("Judge").describe()

In [ ]:
bonds_by_judge.to_csv("../Output/Bond by Judge.csv")

In [ ]:
bonds_by_judge_avg = judge_chart.groupby(["Judge", "Courthouse", "Highest Charge Class"])["Bond"].mean()

In [ ]:
bonds_by_judge_max = judge_chart.groupby(["Judge", "Courthouse", "Highest Charge Class"])["Bond"].max()

In [ ]:
bonds_by_judge_min = judge_chart.groupby(["Judge", "Courthouse", "Highest Charge Class"])["Bond"].min()

In [ ]:
bonds_by_judge_avg

### Other Searches

In [ ]:
missing_df  = pd.read_csv("../Output/Still Unmatched Names.csv", header=None)

In [ ]:
grid = []
for name in list(missing_df[0]):
    row = []
    full = name.split()
    first = full[0]
    last = full[-1]
    row.append(last)
    row.append(first)
    all_peeps = coll.find({"Last Name" : last, "First Name" : first})
    if all_peeps.count() >= 1:
        person = all_peeps[0]
        row.append(person["Bond Court Date"])
        row.append(person["Courthouse"])
        row.append(person["Judge"])
    else:
        full_search = coll.find({"All Spellings": "/" + name + "/"})
        if full_search.count() >=1:
            person = full_search[0]
            row.append(person["Bond Court Date"])
            row.append(person["Courthouse"])
            row.append(person["Judge"])
        else:
            print(name)

    grid.append(row)

In [ ]:
people_plus_courthouse = pd.DataFrame(grid)

In [ ]:
people_plus_courthouse

In [ ]:
people_plus_courthouse.to_csv("../Output/Courthouses and Judges for Missing People.csv", header=None, index = False)

### Close DB Connection

In [ ]:
client.close()